In [16]:
import numpy as np
import pickle
from pandas import DataFrame, read_csv
from sklearn import preprocessing
import warnings
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow import nn 


#warnings.filterwarnings("ignore")

In [45]:
#########################
############ Hyperparamter
#########################

st_dev = 0.2
learning_rate = 0.001
training_iters = 10000000
batch_size = 128
display_step = 100
display_testing_step = 400
dropout_rate = 0.8
#validate_dropout = 0.95

n_input = 10 # MNIST data input (img shape: 28*28)
n_steps = 4 # timesteps
n_hidden = 100 # hidden layer num of features
n_classes = 5 # MNIST total classes (0-9 digits)


In [46]:
#########################
############ Read in data
#########################

data = pickle.load( open( "kdd99.p", "rb" ) )

x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']


In [47]:
tf.reset_default_graph()

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Define weights
W = {
    'out': tf.Variable(tf.truncated_normal([n_hidden, n_classes], stddev=st_dev))
}


b = {
    'out': tf.Variable(tf.zeros([n_classes]))
}

In [48]:
num_layers = 3

def RNN(x, W, b):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    with tf.variable_scope('cell_def'):
        #lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
        lstm_cell = tf.contrib.rnn.LSTMCell(n_hidden, state_is_tuple = True)
        lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, input_keep_prob=keep_prob)
        lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers, state_is_tuple=True)
        lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob) 
        
        
    # Get lstm cell output
    with tf.variable_scope('rnn_def'):
        output, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(output[-1], W['out']) + b['out']


In [49]:

with tf.variable_scope('LSTM1'):
    pred = RNN(x, W, b)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)



# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [50]:
# Initializing the variables
init = tf.global_variables_initializer()


In [51]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        # Generate a minibatch.
        batch_x = x_train[offset:(offset + batch_size), :]
        batch_y = y_train[offset:(offset + batch_size), :]
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob:dropout_rate})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, keep_prob:dropout_rate})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob:dropout_rate})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
        
        if step % display_testing_step == 0:
        
            test_len = 1024
            test_data = x_test[:test_len].reshape((-1, n_steps, n_input))
            test_label = y_test[:test_len]
            print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label, keep_prob:1.0}))
        
    print("Optimization Finished!")



Iter 12800, Minibatch Loss= 1.388641, Training Accuracy= 0.00000
Iter 25600, Minibatch Loss= 0.022735, Training Accuracy= 1.00000
Iter 38400, Minibatch Loss= 0.004888, Training Accuracy= 1.00000
Testing Accuracy: 0.206055
Iter 51200, Minibatch Loss= 0.054580, Training Accuracy= 1.00000
Iter 64000, Minibatch Loss= 0.040695, Training Accuracy= 1.00000
Iter 76800, Minibatch Loss= 0.810280, Training Accuracy= 0.33594
Iter 89600, Minibatch Loss= 0.048719, Training Accuracy= 1.00000
Testing Accuracy: 0.206055
Iter 102400, Minibatch Loss= 0.003845, Training Accuracy= 1.00000
Iter 115200, Minibatch Loss= 0.023924, Training Accuracy= 1.00000
Iter 128000, Minibatch Loss= 0.007077, Training Accuracy= 1.00000
Iter 140800, Minibatch Loss= 3.807020, Training Accuracy= 0.00000
Testing Accuracy: 0.576172
Iter 153600, Minibatch Loss= 0.017740, Training Accuracy= 1.00000
Iter 166400, Minibatch Loss= 0.001326, Training Accuracy= 1.00000
Iter 179200, Minibatch Loss= 0.000772, Training Accuracy= 1.00000
It